In [1]:
!pip install backtesting ta-lib seaborn matplotlib

In [2]:
import numpy as np
import pandas as pd
import talib
from backtesting import Backtest, Strategy

In [3]:
import mysql.connector
from mysql.connector import Error
conn = None
try:
    # Conéctate a la base de datos MySQL
    conn = mysql.connector.connect(
        host='localhost',
        database='backtesting',
        user='pruebas',
        password='contraseñaprueba'
    )

    if conn.is_connected():
        # Ejecuta la consulta para obtener los datos
        query = "SELECT * FROM btc_usdt"
        df_btc = pd.read_sql(query, conn)

        print("Datos obtenidos de la base de datos:")
        print(df_btc.head())

except Error as e:
    print(f"Error: {e}")
finally:
    if conn is not None and conn.is_connected():
        conn.close()
        print("Conexión a MySQL cerrada.")

#para cargar correctamente los datos a la estrategia las columnas deben estar con mayusculas
df_btc.columns = [col.capitalize() for col in df_btc.columns]
df_btc

Datos obtenidos de la base de datos:
            timestamp      open      high       low     close      volume
0 2024-04-25 06:00:00  64223.36  64391.99  64195.70  64288.00   363.60804
1 2024-04-25 07:00:00  64288.00  64379.85  63824.07  64017.99   951.17474
2 2024-04-25 08:00:00  64017.99  64176.86  63708.20  63948.78  1329.27338
3 2024-04-25 09:00:00  63948.78  64192.00  63809.13  63966.02   833.85634
4 2024-04-25 10:00:00  63966.03  63966.03  63337.25  63462.01  2418.87397
Conexión a MySQL cerrada.


/tmp/ipykernel_407703/3340958104.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_btc = pd.read_sql(query, conn)


,Timestamp,Open,High,Low,Close,Volume
0,2024-04-25 06:00:00,64223.36,64391.99,64195.70,64288.00,363.60804
1,2024-04-25 07:00:00,64288.00,64379.85,63824.07,64017.99,951.17474
2,2024-04-25 08:00:00,64017.99,64176.86,63708.20,63948.78,1329.27338
3,2024-04-25 09:00:00,63948.78,64192.00,63809.13,63966.02,833.85634
4,2024-04-25 10:00:00,63966.03,63966.03,63337.25,63462.01,2418.87397
...,...,...,...,...,...,...
995,2024-06-05 17:00:00,71312.78,71704.00,71231.48,71628.98,1367.90189
996,2024-06-05 18:00:00,71628.98,71748.22,70895.15,71096.81,1778.53896
997,2024-06-05 19:00:00,71096.80,71398.73,70871.20,71279.95,937.98536
998,2024-06-05 20:00:00,71279.95,71357.68,71160.00,71196.02,699.46084


In [4]:
class SMAPullbackStrategy(Strategy):
    n = 20

    def init(self):
        close = self.data.Close
        self.sma = self.I(talib.SMA, close, self.n)
    
    def next(self):
        price = self.data.Close[-1]
        sma = self.sma[-1]
        yesterdays_low = self.data.Low[-1]
        stop_loss = yesterdays_low
        take_profit = yesterdays_low + 1.5 *(price - stop_loss)

        if self.position.is_long:
            if price <= stop_loss or price >= take_profit:
                self.position.close()

        elif price > sma:
            print(stop_loss, ", ", take_profit)
            try:
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass
        

        

In [6]:
bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)
stats = bt.run()
#bt.plot()

64304.88 ,  64597.425
64294.78 ,  64424.47
64000.02 ,  64830.975
63685.94 ,  64907.045
62750.0 ,  63749.315
63552.68 ,  64148.66
63932.78 ,  64066.04000000001
63827.88 ,  63850.770000000004
63810.0 ,  64025.985
63800.0 ,  64050.71
63894.21 ,  63926.145000000004
63639.17 ,  63745.38500000001
63676.0 ,  63800.995
63345.55 ,  63868.975
63534.0 ,  63896.145000000004
63483.18 ,  63816.479999999996
63570.3 ,  63867.555
63694.5 ,  64038.735
61878.61 ,  63313.17999999999
63041.04 ,  64314.479999999996
63688.04 ,  65025.979999999996
63560.5 ,  63628.765
62995.7 ,  64173.51500000001
57331.1 ,  59705.435
57828.0 ,  58633.455
57404.08 ,  57740.799999999996
57618.07 ,  57712.58500000001
57612.73 ,  57945.865
57626.89 ,  57893.095
57561.14 ,  57921.545000000006
57783.04 ,  58947.97
58446.18 ,  58611.795
58222.0 ,  58564.33
58300.0 ,  59432.875
58972.44 ,  59410.005000000005
58943.38 ,  59054.62
58955.31 ,  59307.705
58987.49 ,  59581.534999999996
58637.62 ,  58772.36499999999
58701.1 ,  59318.320000

/tmp/ipykernel_407703/2355427670.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SMAPullbackStrategy, cash=1000000, commission=0.002)


In [7]:
print(stats)

Start                                     0.0
End                                     999.0
Duration                                999.0
Exposure Time [%]                        32.8
Equity Final [$]                 750210.54256
Equity Peak [$]                     1000000.0
Return [%]                         -24.978946
Buy & Hold Return [%]               10.497013
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.078156
Avg. Drawdown [%]                  -25.078156
Max. Drawdown Duration                  976.0
Avg. Drawdown Duration                  976.0
# Trades                                133.0
Win Rate [%]                        66.165414
Best Trade [%]                       1.157335
Worst Trade [%]                     -2.881003
Avg. Trade [%]                    